In [5]:
import geopandas
import pandas as pd
import numpy as np
from gravity_utils import *

In [33]:
def get_building_sams(buildings, zones):
    i = 0
    for zone_geom, index in zip(zones.geometry, zones.index):
        in_sams = buildings.geometry.within(zone_geom) # pandas series of booleans. True if within.
        buildings.loc[in_sams, 'zone'] = index
#         print(i)
        i = i + 1

In [7]:
od_matrix = pd.read_csv('data/od_matrix.csv', sep=';', index_col=0)

In [8]:
od_matrix.columns = od_matrix.columns.astype(int)

In [9]:
zone = od_matrix.loc[12700005, :]

In [10]:
b1_sams = geopandas.read_file('data/GIS/B1_samsSW_20131231_shp/B1_sw_region.shp')
b1_sams = b1_sams.loc[(b1_sams.KOMMUN.astype(int) < 1300) & (b1_sams.KOMMUN.astype(int) >= 1200)]
b1_sams = b1_sams.to_crs(epsg=4326)

In [11]:
df = []
for kommun in b1_sams.KOMMUN.unique():
    df.append((kommun, b1_sams.set_index('KOMMUN').loc[kommun].unary_union))
municipalities = geopandas.GeoDataFrame(df, columns=['municipality', 'geometry'])
municipalities = municipalities.loc[(municipalities.municipality.astype(int) != 1265) & (municipalities.municipality.astype(int) != 1270)]
municipalities = municipalities.set_index('municipality')
municipalities['zone'] = np.NaN

In [12]:
municipalities.head()

,geometry,zone
municipality,,
1280,"POLYGON ((13.10703847278691 55.52965140966583,...",NaN
1233,"POLYGON ((12.96815916315039 55.33841434999049,...",NaN
1283,"POLYGON ((12.91409022440169 55.99192642553837,...",NaN
1282,"POLYGON ((12.96068143061406 55.86083486092289,...",NaN
1284,"POLYGON ((12.68175018721998 56.17075423403756,...",NaN


In [13]:
a2_ruta = geopandas.read_file('data/GIS/A2_ruta_2013/A2.shp')
a2_ruta['geometry'] = a2_ruta.centroid
get_building_sams(a2_ruta, municipalities)

,Rutstorl,Ruta,Offentliga,Naringsliv,Totalt,geometry
0,None,3500006223750,3,0,3,POINT (12.58815058442633 56.13634879163538)
1,None,3497506225250,28,82,110,POINT (12.58328632274202 56.14973530004571)
2,None,3490006225500,11,30,41,POINT (12.57108336326716 56.15174288403292)
3,None,3490006225250,10,34,44,POINT (12.57122499527641 56.14949873024521)
4,None,3500006225000,20,50,70,POINT (12.58744750397297 56.14756971428457)


In [31]:
a4_ruta = geopandas.read_file('data/GIS/A4_ruta_2013/A4.shp')
a4_ruta['geometry'] = a4_ruta.centroid
get_building_sams(a4_ruta, municipalities)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [32]:
sams_zones = b1_sams.loc[(b1_sams.KOMMUN.astype(int) == 1265) | (b1_sams.KOMMUN.astype(int) == 1270)]
sams_zones = sams_zones.set_index('SAMSCODE')
get_building_sams(a2_ruta, sams_zones)
get_building_sams(a4_ruta, sams_zones)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36


In [34]:
a2_ruta.zone.value_counts(dropna=False).head()

NaN     6588
1290    1850
1293    1590
1280    1058
1283    1032
Name: zone, dtype: int64

In [36]:
a4_ruta.zone.value_counts(dropna=False).head()

NaN     4620
1290    1466
1280    1279
1293    1167
1283    1074
Name: zone, dtype: int64

In [38]:
a2_ruta.dropna(axis=0, subset=['zone']).to_csv('data/GIS/a2_ruta_zone_codes.csv', sep=';')
a4_ruta.dropna(axis=0, subset=['zone']).to_csv('data/GIS/a4_ruta_zone_codes.csv', sep=';')

In [39]:
a2_ruta.head()

,Rutstorl,Ruta,Offentliga,Naringsliv,Totalt,geometry,zone
0,None,3500006223750,3,0,3,POINT (12.58815058442633 56.13634879163538),1283
1,None,3497506225250,28,82,110,POINT (12.58328632274202 56.14973530004571),1284
2,None,3490006225500,11,30,41,POINT (12.57108336326716 56.15174288403292),1284
3,None,3490006225250,10,34,44,POINT (12.57122499527641 56.14949873024521),1284
4,None,3500006225000,20,50,70,POINT (12.58744750397297 56.14756971428457),1284
